![Basic Test](images/header_workflow.png)

##  ML y SoC-FPGA: Hardware verification


### Import local libraries

In [ ]:
import ipyparallel as ipp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import hyperfpga_cluster as hfc


### Initialize the node and deploy the selected firmware

In [ ]:
nodes = hfc.get_nodes()
test_node = nodes[0]; print(test_node)
cluster = hfc.HyperFPGACluster(nodes=test_node, firmware="ML_4eg_21-KalEdge", update=False , n_engines=1)#len(test_node))
cluster.create_profile()
await cluster.configure()

### Init remote cluster

In [ ]:
rc = cluster.start_and_connect_sync()
dview = rc[:]

### Import the libraries required for remote execution

In [ ]:
dview.execute("""
from comblock import Comblock
cb = Comblock()
""")

### Function for remote FPGA execution
This function sends the input data to the target device and retrieves the inference results.


In [ ]:
# Execute the function in parallel using ipyparallel
@dview.remote(block=True)  # Changed to block=False

def inference(data_in):
    from time import sleep

    cb.fifo_out_clear()
  
    # Init HLS IP core for ML classification
    cb.write_reg(0, 1)
    cb.write_reg(1, 1)
   #cb.write_reg(1, 0)

    for val in data_in:
        cb.write_fifo(val)
        
    sleep(0.1)

    # Read inference result from the correpsondig register
    a = cb.read_reg(0)    
   
    return a

### Load test dataset

In [ ]:
# Load the raw test dataset.
# The CSV file has no header row, and the last column contains the class labels
# except for a row where the label appears as the string "class" (likely a header artifact).
df = pd.read_csv("datasets/test.csv", header=None)

# Remove any rows where the last column contains the string "class".
# This cleans up header-like artifacts inside the file.
df = df[df.iloc[:, -1] != "class"].reset_index(drop=True)

# Split into features (all columns except last) and labels (last column).
# Convert to int32 so that data types match the FPGA input format.
X = df.iloc[:, :-1].astype(np.int32).to_numpy()
y_true = df.iloc[:, -1].astype(np.int32).to_numpy()

# Get dataset dimensions for verification.
num_samples = X.shape[0]
num_features = X.shape[1]

print("Samples:", num_samples)
print("Features:", num_features)


### Remote Inference Execution and Prediction Mapping

In [ ]:
res = []
y_true_loop = []

for k in range(0, num_samples):
    signal = X[k].tolist()

    #plt.figure(figsize=(13,2))
    #plt.plot(signal)
    #plt.title(f"Input signal k={k}")
    #plt.xlabel("Samples")
    #plt.ylabel("Amplitude (ADC units)")
    #plt.show()

    pred = inference(signal)

    # unwrap
    if isinstance(pred, (list, tuple)) and len(pred) == 1:
        pred_code = int(pred[0])
    else:
        import numpy as np
        pred = np.asarray(pred).reshape(-1)
        if pred.size == 1:
            pred_code = int(pred[0])
        else:
            raise ValueError(f"Unexpected inference output (vector): {pred}")

    y_true_loop.append(int(y_true[k]))

    if pred_code == 0:
        res.append(0)
    elif pred_code == 1:
        res.append(1)
    else:
        raise ValueError(f"Unexpected scalar output: {pred_code}")


### Compute and Plot the Confusion Matrix

In [ ]:
y_true_arr = np.array(y_true_loop, dtype=int)
y_pred_arr = np.array(res, dtype=int)

# -----------------------------
# 5) Confusion matrix (no sklearn)
# -----------------------------
TP = int(np.sum((y_true_arr == 1) & (y_pred_arr == 1)))
TN = int(np.sum((y_true_arr == 0) & (y_pred_arr == 0)))
FP = int(np.sum((y_true_arr == 0) & (y_pred_arr == 1)))
FN = int(np.sum((y_true_arr == 1) & (y_pred_arr == 0)))

cm = np.array([[TN, FP],
               [FN, TP]])

print("\nConfusion Matrix (rows=true, cols=pred)")
print(cm)
print("Total:", cm.sum())

# -----------------------------
# 6) Metrics
# -----------------------------
accuracy  = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) else 0.0
precision = TP / (TP + FP) if (TP + FP) else 0.0
recall    = TP / (TP + FN) if (TP + FN) else 0.0
f1        = (2 * precision * recall / (precision + recall)) if (precision + recall) else 0.0

print(f"\nAccuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")

# -----------------------------
# 7) Plot confusion matrix (no sklearn)
# -----------------------------
import matplotlib.pyplot as plt

plt.figure(figsize=(4,4))
plt.imshow(cm, cmap="Blues")

plt.xticks([0,1], ["Pred 0", "Pred 1"])
plt.yticks([0,1], ["True 0", "True 1"])

for i in range(2):
    for j in range(2):
        plt.text(j, i, cm[i, j],
                 ha="center", va="center",
                 color="white" if cm[i,j] > cm.max()/2 else "black",
                 fontsize=14)


plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()

In [ ]:
accuracy  = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) else 0
recall    = TP / (TP + FN) if (TP + FN) else 0
f1        = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")


----

In [ ]:
cluster.clean_cluster()
rc.shutdown()